In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path= '/content/drive/MyDrive/DATA | Unigap /PYTHON/PROJECT PYTHON/Project 2'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
payment_report = pd.read_csv(path+'/payment_report.csv')
transactions = pd.read_csv(path+'/transactions.csv')
product = pd.read_csv(path+'/product.csv')
print(payment_report.head())
print(transactions.head())
print(product.head())

  report_month payment_group  product_id  source_id     volume
0      2023-01       payment          12         45  624110375
1      2023-01       payment          17         45  335715113
2      2023-01       payment          18         45  737784466
3      2023-01       payment          19         45  120963069
4      2023-01       payment          20         45  319653158
   transaction_id  merchant_id   volume  transType  transStatus   sender_id  \
0      3002692434            5   100000         24            1  10199794.0   
1      3002692437          305    20000          2            1  14022211.0   
2      3001960110         7255    48605         22            1         NaN   
3      3002680710         2270  1500000          2            1  10059206.0   
4      3002680713         2275    90000          2            1  10004711.0   

   receiver_id extra_info      timeStamp  
0     199794.0        NaN  1682932054455  
1   14022211.0        NaN  1682932054912  
2   10530940.0    

**EDA**

Check each column: missing data? duplicates? incorrect data types?

Summarize numerical data: any incorrect values?


In [ ]:
#Merge payment_report.csv with product.csv
payment_enriched=payment_report.merge(product, on='product_id', how='left')
#Kiểm tra dữ liệu tổng quan
payment_enriched.info()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 50.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  
 2   volume          1324002 non-null  int64  
 3   transType       1324002 non-null  int64  
 4   transStatus     1324002 non-null  int64  
 5   sender_id   

Nhận xét:
Payment_enriched

1.Category và Team_own có 22 null rows ->  có thể thay thế bằng "Unknown"
2. Các kiểu dữ liệu có vẻ hợp lý

Transactions
1. sender_id có 49059 null rows (~3.7%) -> ignore
2. receiver_id có 164,795 null rows (12.4%)-> ignore
3. extra_info hầu hết bị thiếu gía trị -> Hành động: xoá nguyên column extra_info
4. Dữ liệu của timeStamp đang có định dạng int64-> Hành động: nên đưa về định dạng thời gian
5. sender_id, receiver đang có định dang float64 -> Hành động: nên đưa về định dạng int64

In [ ]:
#Kiểm tra chính xác dữ liệu null
print(payment_enriched.isnull().sum())
print(transactions.isnull().sum())

report_month      0
payment_group     0
product_id        0
source_id         0
volume            0
category         22
team_own         22
dtype: int64
transaction_id          0
merchant_id             0
volume                  0
transType               0
transStatus             0
sender_id           49059
receiver_id        164795
extra_info        1317907
timeStamp               0
dtype: int64


In [ ]:
#Kiểm tra dữ liệu trùng lặp
duplicates_payment = payment_enriched.duplicated().sum()
duplicates_transactions = transactions.duplicated().sum()
print('Số lượng dòng bị duplicate trong payment_enrich là:' + str(duplicates_payment))
print('Số lượng dòng bị duplicate trong transactions là:'+ str(duplicates_transactions))

Số lượng dòng bị duplicate trong payment_enrich là:0
Số lượng dòng bị duplicate trong transactions là:28


Nhận xét:
Xoá 28 dòng bị duplicate trong bảng transactions

In [ ]:
#Kiểm tra giá trị không hợp lệ
invalid_values_payment = (payment_enriched["volume"] < 0).sum()
invalid_values_transactions = (transactions["volume"] < 0).sum()
print(invalid_values_payment)
print(invalid_values_transactions)

0
0


Nhận xét: Không có giá trị nào không hợp lệ

# Part II: Data Wrangling


In [ ]:
#Câu 1:Top 3 product_ids with the highest volume.
payment_enriched.groupby("product_id")["volume"].sum().sort_values(ascending=False).head(3)

,volume
product_id,
1976,61797583647
429,14667676567
372,13713658515


In [ ]:
#Câu 2: Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?
product_team_count = payment_enriched.groupby('product_id')['team_own'].nunique()
abnormal_products=product_team_count[product_team_count>1].sum()
print(abnormal_products)

0


Câu 2: Không có sản phẩm nào vi phạm quy tắc mỗi sản phẩm chỉ thuộc 1 team

In [ ]:
#Câu 3:Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.

#Chọn các đơn hàng từ Quý 2 năm 2023 (Tháng 4 năm 2023)
payment_q2_2023 = payment_enriched[payment_enriched['report_month']>='2023-04']
#Tìm team có lowest performance
team_lowest_performance = payment_enriched.groupby('team_own')['volume'].sum().sort_values(ascending=True).head(1)

print(team_lowest_performance)

team_own
APS    272891202
Name: volume, dtype: int64


In [ ]:
#Tìm category đóng góp ít nhất trong team đó
category_lowest_performance = payment_q2_2023[payment_q2_2023['team_own']=='APS'].groupby('category')['volume'].sum().sort_values(ascending=True).head(1)
print(team_lowest_performance)
print(category_lowest_performance)

team_own
APS    272891202
Name: volume, dtype: int64
category
PXXXXXE    25232438
Name: volume, dtype: int64


Câu 3:
Team "APS" có hiệu suất thấp nhất.
Category đóng góp ít nhất cho team APS: "PXXXXXE"

In [ ]:
#Câu 4: Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?
refund_contribution= payment_enriched[payment_enriched['payment_group']=='refund'].groupby('source_id')['volume'].sum()
top_highest_refund_contribution= refund_contribution.idxmax()
print(top_highest_refund_contribution)

38


Câu 4: Source_id có đóng góp cao nhất trong refund transactions là 38

# Using transactions.csv
Câu 5: Define type of transactions (‘transaction_type’) for each row, given:
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, `merchant_id` = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions


In [ ]:
#Xác định mỗi loại giao dịch
def classify_transaction(row):
  if row['transType'] == 2:
    if row['merchant_id'] == 1205:
      return "Bank Transfer Transaction"
    elif row['merchant_id'] == 2269:
      return "Withdraw Money Transaction"
    elif row['merchant_id'] == 2270:
      return "Top Up Money Transaction"
    else:
      return " Payment Transaction"
  elif row['transType'] == 8:
    if row['merchant_id'] ==2250:
      return "Transfer Money Transaction"
    else:
      return "Split Bill Transaction"
  else:
    return "Invalid Transaction"

# Áp dụng phân loại giao dịch
transactions["transaction_type"] = transactions.apply(classify_transaction, axis=1)


In [ ]:
# Câu 6: Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.
#Loại bỏ giao dịch Invalid
valid_transactions = transactions[transactions["transaction_type"] != "Invalid Transaction"]
#Tính số giao dịch, tổng volume, senders và receivers
transaction_summary= valid_transactions.groupby('transaction_type').agg(
          transaction_count =('transaction_id','count'),
          volume = ('volume','sum'),
          unique_senders = ('sender_id','nunique'),
          unique_receivers = ('receiver_id','nunique')
).reset_index()

transaction_summary.head()


,transaction_type,transaction_count,volume,unique_senders,unique_receivers
0,Payment Transaction,432402,95269696601,147670,124846
1,Bank Transfer Transaction,37879,50605806190,23156,9271
2,Split Bill Transaction,1376,4901464,1323,572
3,Top Up Money Transaction,290502,108606478829,110409,110409
4,Transfer Money Transaction,341177,37033171492,39021,34585
